In [187]:
import pandas as pd
from predictor import Predictor
from datetime import datetime
import mlflow
import numpy as np
from scipy import stats
import joblib
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error


## Run predictions on test set simulation to simulate prediciton of new data

RUN this first
```bash
docker run -p 8002:8080 xgboost_dev
```



In [188]:
def convert_date(x):
    return datetime(x['year'], x['month'], 1)

def convert_remaining_lease(x):
    # example: "60 years 11 months"
    try:
        years = int(x.split(' ')[0])
        months = int(x.split(' ')[2])
    except IndexError:
        # print(x)
        return years * 12
    total_months = years * 12 + months
    return total_months

In [189]:
# take in a part of the actual data we have, run predictions for these then store everything
# (features, pred, actual, time) somewhere for me to use

# 1. get the portion of the actual data 
# simulate pulling from data wareshouse 
df = pd.read_csv("../../data/resale_flats_transformed.csv")

# 2. shape it to something like example.csv 
input_df = pd.read_csv("example.csv")
# need to get remaining_lease_years, remaining_lease_months
df['remaining_lease_months'] = df['remaining_lease'].apply(convert_remaining_lease)
df['remaining_lease_years'] = 0
# get sale date
df['sale_date'] = df[['year', 'month']].apply(convert_date, axis=1)
# separate to test set 
df = df[df['sale_date'] >= datetime(2024, 4, 1)]

# get the columns that are in the input_df (example.csv)
df = df[input_df.columns]

# 3. run predictions on this data
predictor = Predictor(port=8002)

# # 4. store the results in a csv file
# this takes quite long 
pred_df = predictor.predict_df(df)
pred_df.to_csv("predictions.csv", index=False)

Predicted price: $312006.14
Predicted price: $319844.87
Predicted price: $298010.00
Predicted price: $340841.20
Predicted price: $302989.14
Predicted price: $360368.95
Predicted price: $360368.95
Predicted price: $396790.31
Predicted price: $368611.54
Predicted price: $552646.87
Predicted price: $243149.41
Predicted price: $356433.75
Predicted price: $335436.49
Predicted price: $350965.10
Predicted price: $313708.16
Predicted price: $329097.64
Predicted price: $356433.75
Predicted price: $350965.10
Predicted price: $350965.10
Predicted price: $352232.44
Predicted price: $328215.28
Predicted price: $306261.32
Predicted price: $349207.71
Predicted price: $338278.44
Predicted price: $344760.02
Predicted price: $347995.26
Predicted price: $361181.82
Predicted price: $344191.43
Predicted price: $331038.17
Predicted price: $352140.65
Predicted price: $329535.27
Predicted price: $309671.72
Predicted price: $356907.93
Predicted price: $503646.44
Predicted price: $435770.64
Predicted price: $39

python -m mlflow server --host 127.0.0.1 --port 8000

## Simulate collection of ground truth some time in future

In [190]:
df = pd.read_csv("predictions.csv")
df

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479
...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438


In [191]:
# randomly generate actual prices based on some formula of the predicted price
# this is to simulate the actual data we have
actual_prices = []
for i, row in df.iterrows():
    pred_price = row['predicted_price']
    actual_price = pred_price + np.random.normal(0, 20000)
    actual_prices.append(actual_price)

df['actual_price'] = actual_prices
df

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,342364.226885
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,347023.581691
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,291062.922014
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,313909.863342
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,282156.321539
...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,414531.300252
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,360545.476362
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,329407.143159
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,580976.865536


In [192]:
df.to_csv("predictions_with_ground_truth.csv", index=False)

## Monitoring

### ground truth evaluation

In [193]:
# check model history performance in mlflow through code
mlflow.set_tracking_uri('http://localhost:8000')
# Get the model history
model_history = mlflow.search_runs()
model_history

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MSE,metrics.MAPE,metrics.MAE,params.reg_lambda,...,params.reg_alpha,params.max_depth,params.subsample_freq,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.type,tags.model,tags.mlflow.user,tags.feature
0,95525df54dc94480badb2c2e398f602c,0,FINISHED,mlflow-artifacts:/0/95525df54dc94480badb2c2e39...,2024-04-11 07:04:45.180000+00:00,2024-04-11 07:04:57.013000+00:00,0.141749,0.873252,0.260327,0.0,...,0.0,-1,0,"[{""run_id"": ""95525df54dc94480badb2c2e398f602c""...",c:\Users\valen\anaconda3\lib\site-packages\ipy...,enchanting-dog-18,LOCAL,LightGBM,valen,Resale Price Prediction
1,bfa21874d08f4798a41d7ab63ae82e5c,0,FINISHED,mlflow-artifacts:/0/bfa21874d08f4798a41d7ab63a...,2024-04-11 07:04:20.464000+00:00,2024-04-11 07:04:45.131000+00:00,0.072370,0.790721,0.198472,None,...,None,None,None,"[{""run_id"": ""bfa21874d08f4798a41d7ab63ae82e5c""...",c:\Users\valen\anaconda3\lib\site-packages\ipy...,debonair-vole-478,LOCAL,XGBoost,valen,Resale Price Prediction
2,2cc3764d90ef4609b4e57fcff522004e,0,FINISHED,mlflow-artifacts:/0/2cc3764d90ef4609b4e57fcff5...,2024-03-29 15:37:34.169000+00:00,2024-03-29 15:37:39.549000+00:00,0.131307,0.833802,0.250773,0.0,...,0.0,-1,0,"[{""run_id"": ""2cc3764d90ef4609b4e57fcff522004e""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,chill-swan-395,LOCAL,LightGBM,ivankoh,Resale Price Prediction
3,08f5fe0400f649b283b222fa850dc091,0,FINISHED,mlflow-artifacts:/0/08f5fe0400f649b283b222fa85...,2024-03-29 15:37:22.144000+00:00,2024-03-29 15:37:34.144000+00:00,0.070854,0.782482,0.196303,None,...,None,None,None,"[{""run_id"": ""08f5fe0400f649b283b222fa850dc091""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,resilient-cub-967,LOCAL,XGBoost,ivankoh,Resale Price Prediction
4,a4e061e14928418cbce808a5b90462ec,0,FINISHED,mlflow-artifacts:/0/a4e061e14928418cbce808a5b9...,2024-03-29 13:42:44.513000+00:00,2024-03-29 13:42:49.864000+00:00,0.131307,0.833802,0.250773,0.0,...,0.0,-1,0,"[{""run_id"": ""a4e061e14928418cbce808a5b90462ec""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,omniscient-toad-552,LOCAL,LightGBM,ivankoh,Resale Price Prediction
5,7eba7d2fd2ae4f019ce155ba1338b95b,0,FINISHED,mlflow-artifacts:/0/7eba7d2fd2ae4f019ce155ba13...,2024-03-29 13:42:31.960000+00:00,2024-03-29 13:42:44.467000+00:00,0.070854,0.782482,0.196303,None,...,None,None,None,"[{""run_id"": ""7eba7d2fd2ae4f019ce155ba1338b95b""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,casual-mule-437,LOCAL,XGBoost,ivankoh,Resale Price Prediction
6,b44e97e862f145f19a1f1838f7a79610,0,FINISHED,mlflow-artifacts:/0/b44e97e862f145f19a1f1838f7...,2024-03-29 13:22:25.378000+00:00,2024-03-29 13:22:30.743000+00:00,0.131307,0.833802,0.250773,0.0,...,0.0,-1,0,"[{""run_id"": ""b44e97e862f145f19a1f1838f7a79610""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,industrious-carp-854,LOCAL,LightGBM,ivankoh,Resale Price Prediction
7,68279395fa044d798337f962a4600287,0,FINISHED,mlflow-artifacts:/0/68279395fa044d798337f962a4...,2024-03-29 13:22:14.933000+00:00,2024-03-29 13:22:25.325000+00:00,0.070854,0.782482,0.196303,None,...,None,None,None,"[{""run_id"": ""68279395fa044d798337f962a4600287""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,masked-bat-604,LOCAL,XGBoost,ivankoh,Resale Price Prediction
8,709239b403a44bef87a88a17ef5f4890,0,FINISHED,mlflow-artifacts:/0/709239b403a44bef87a88a17ef...,2024-03-29 13:20:52.053000+00:00,2024-03-29 13:21:05.995000+00:00,0.070854,0.782482,0.196303,None,...,None,None,None,"[{""run_id"": ""709239b403a44bef87a88a17ef5f4890""...",/Users/ivankoh/.local/lib/python3.9/site-packa...,popular-gnat-419,LOCAL,XGBoost,ivankoh,Resale Price Prediction
9,24f757752aa04c9380df54f2a0bd2695,0,FAILED,mlflow-artifacts:/0/24f757752aa04c9380df54f2a0...,2024-03-29 13:14:35.744000+00:00,2024-03-29 13:14:43.578000+00:00,0.070854,0.782482,0.196303,None,...,None,None,None,None,/Users/ivankoh/.local/lib/python3.9/site-packa...,red

In [194]:
def getmodelhist(model_name):
    model_history = mlflow.search_runs()
    model_history = model_history[model_history["tags.model"] == model_name]

    mean_mse = model_history["metrics.MSE"].mean()
    mean_mae = model_history["metrics.MAE"].mean()
    mean_mape = model_history["metrics.MAPE"].mean()

    return {
        "model_name": model_name,
        "metrics" : {
            "mean_mse": mean_mse,
            "mean_mae": mean_mae,
            "mean_mape": mean_mape
        }
    }



In [195]:
getmodelhist("XGBoost")

{'model_name': 'XGBoost',
 'metrics': {'mean_mse': 0.07110626937122871,
  'mean_mae': 0.19666476107049982,
  'mean_mape': 0.7838551752810231}}

In [196]:
getmodelhist("LightGBM")

{'model_name': 'LightGBM',
 'metrics': {'mean_mse': 0.1339175667494985,
  'mean_mae': 0.2531615249495469,
  'mean_mape': 0.8436645015752049}}

In [197]:
df_gt = pd.read_csv("predictions_with_ground_truth.csv")
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,342364.226885
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,347023.581691
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,291062.922014
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,313909.863342
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,282156.321539
...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,414531.300252
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,360545.476362
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,329407.143159
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,580976.865536


In [198]:
# import scaler
TARGET_SCALER_PATH = '_scalers/target_scaler.save'
target_scaler = joblib.load(TARGET_SCALER_PATH)

In [199]:
df_gt["actual_price_scaled"] = target_scaler.transform(df_gt['actual_price'].values.reshape(-1, 1))
df_gt["predicted_price_scaled"] = target_scaler.transform(df_gt['predicted_price'].values.reshape(-1, 1))

In [200]:
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease_months,flat_type,storey_range,flat_model,district,predicted_price,actual_price,actual_price_scaled,predicted_price_scaled
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,342364.226885,-0.812781,-0.996796
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,347023.581691,-0.784538,-0.949282
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,291062.922014,-1.123743,-1.081633
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,313909.863342,-0.985257,-0.822013
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,282156.321539,-1.177730,-1.051453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,414531.300252,-0.375342,-0.331979
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,360545.476362,-0.702576,-0.692508
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,329407.143159,-0.891320,-0.806234
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,580976.865536,0.633566,0.796600


In [201]:
# Get performance metrics of new test data after ground truth collected
mape_new = mean_absolute_percentage_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])
mae_new = mean_absolute_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])
mse_new = mean_squared_error(df_gt['actual_price_scaled'], df_gt['predicted_price_scaled'])

# Get similar performance metrics during training
mape_train = getmodelhist("XGBoost")['metrics']['mean_mape']
mae_train = getmodelhist("XGBoost")['metrics']['mean_mae']
mse_train = getmodelhist("XGBoost")['metrics']['mean_mse']

In [202]:
# print Train and new, mae,mse,mape
print(f"Train MAPE: {mape_train}, New MAPE: {mape_new}")
print(f"Train MAE: {mae_train}, New MAE: {mae_new}")
print(f"Train MSE: {mse_train}, New MSE: {mse_new}")

Train MAPE: 0.7838551752810231, New MAPE: 1.433314642815224
Train MAE: 0.19666476107049982, New MAE: 0.09657982308425701
Train MSE: 0.07110626937122871, New MSE: 0.014454876553311761


In [203]:
# check if deviation by more than 5 percentage points then notify the team
if mape_new > mape_train + 0.05:
    # notify team
    print(f"MAPE of new test data is {100* (mape_train - mape_new)} percentage points lower than training data")
# if 5% drop in mae
if (mae_new - mae_train)/ mae_train < -0.05:
    # notify team
    print(f"MAE of new test data is {mae_train - mae_new} lower than training data")
# if 5% drop in mse
if mse_new > mse_train + 0.05:
    # notify team
    print(f"MSE of new test data is {mse_train - mse_new} lower than training data")

MAPE of new test data is -64.94594675342007 percentage points lower than training data
MAE of new test data is 0.10008493798624281 lower than training data


In [204]:
df_gt = df_gt.rename(columns={"remaining_lease_months": "remaining_lease"})
df_gt

,year,month,floor_area_sqm,remaining_lease_years,remaining_lease,flat_type,storey_range,flat_model,district,predicted_price,actual_price,actual_price_scaled,predicted_price_scaled
0,2024,4,67.0,0,653,3 ROOM,01 TO 03,New Generation,20.0,312006.144180,342364.226885,-0.812781,-0.996796
1,2024,4,73.0,0,637,3 ROOM,10 TO 12,New Generation,20.0,319844.871643,347023.581691,-0.784538,-0.949282
2,2024,4,68.0,0,666,3 ROOM,01 TO 03,New Generation,20.0,298010.003950,291062.922014,-1.123743,-1.081633
3,2024,4,94.0,0,1050,4 ROOM,19 TO 21,Model A,20.0,340841.201072,313909.863342,-0.985257,-0.822013
4,2024,4,93.0,0,1119,4 ROOM,01 TO 03,Model A,20.0,302989.136479,282156.321539,-1.177730,-1.051453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2024,4,112.0,0,908,5 ROOM,01 TO 03,Improved,27.0,421685.110100,414531.300252,-0.375342,-0.331979
512,2024,4,112.0,0,1088,5 ROOM,13 TO 15,Improved,27.0,362206.359504,360545.476362,-0.702576,-0.692508
513,2024,4,112.0,0,1114,5 ROOM,07 TO 09,Improved,27.0,343444.312940,329407.143159,-0.891320,-0.806234
514,2024,4,181.0,0,807,EXECUTIVE,07 TO 09,Apartment,27.0,607873.703438,580976.865536,0.633566,0.796600


### input drift - feature / label drift and concept drift

https://dataaspirant.com/kolmogorov-smirnov-test/
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [206]:
df_train = pd.read_csv("resale_flats_transformed_train.csv")
df_train.head(5)

,month,flat_type,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,year,lat,lon,district,sale_date
0,4,3 ROOM,01 TO 03,67.0,New Generation,653,380000.0,2024,1.366980,103.855718,20.0,2024-04-01
1,4,3 ROOM,10 TO 12,73.0,New Generation,637,465000.0,2024,1.367487,103.849652,20.0,2024-04-01
2,4,3 ROOM,01 TO 03,68.0,New Generation,666,370000.0,2024,1.381562,103.840002,20.0,2024-04-01
3,4,4 ROOM,19 TO 21,94.0,Model A,1050,882888.0,2024,1.365483,103.844085,20.0,2024-04-01
4,4,4 ROOM,01 TO 03,93.0,Model A,1119,765000.0,2024,1.368864,103.833954,20.0,2024-04-01


### label drift

In [207]:
# label drift
# check if the distribution of the target variable has changed for df_train and df_gt

target_new = df_gt['actual_price']
target_train = df_train['resale_price']
stats.kstest(target_new, target_train)

KstestResult(statistic=0.4020062548911633, pvalue=1.6533077839325588e-75, statistic_location=449263.133805457, statistic_sign=1)

### feature drift

In [208]:
# feature drift - continuous
# check if the distribution of the features has changed for df_train and df_gt

num_cols = ['floor_area_sqm', 'remaining_lease']


for col in num_cols:
    print(col, stats.kstest(df_gt[col], df_train[col]))

floor_area_sqm KstestResult(statistic=0.043185933320604386, pvalue=0.2848595750286168, statistic_location=109.0, statistic_sign=1)
remaining_lease KstestResult(statistic=0.08256912921051834, pvalue=0.001698217067529511, statistic_location=769, statistic_sign=1)


In [218]:
# feature drift - categorical
cat_cols = ['flat_type', 'storey_range', 'flat_model', 'district']

# df_train sample
df_train_sample = df_train.sample(frac=0.2)

for col in cat_cols:
    df_gt_vc = df_gt[col].value_counts()
    df_train_vc = df_train_sample[col].value_counts()
    # ensure both value counts table have the same categories, if any missing then add the category with 0 count
    for cat in df_train_vc.index.unique():
        if cat not in df_gt_vc.index.unique():
            df_gt_vc.loc[cat] = 0
    for cat in df_gt_vc.index.unique():
        if cat not in df_train_vc.index.unique():
            df_train_vc.loc[cat] = 0

    # calculate the frequencies for both value counts table
    df_gt_vc = df_gt_vc / df_gt_vc.sum()
    df_train_vc = df_train_vc / df_train_vc.sum()
     
    print(col, stats.chisquare(df_gt_vc, df_train_vc))

flat_type Power_divergenceResult(statistic=0.02053269249430759, pvalue=0.9999998210410729)
storey_range Power_divergenceResult(statistic=0.023546284983595095, pvalue=1.0)
flat_model Power_divergenceResult(statistic=0.04697196836937345, pvalue=1.0)
district Power_divergenceResult(statistic=0.04218464292203317, pvalue=1.0)


## Feedback